In [1]:
import pandas as pd
import ccxt
from binance.client import Client
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib
from datetime import datetime, timedelta
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
from backtesting.test import SMA


In [2]:
API_KEY = 'Your_key'
API_SECRET = 'Your_key'
client = Client(API_KEY,API_SECRET)

In [3]:
#Creating DataFrame using Binance, data of BTC/USDT

# exchange = ccxt.binance()
symbol = 'ETH/USDT'
timeframe = '4h'
ohlcv = client.get_historical_klines('ETHUSDT',Client.KLINE_INTERVAL_4HOUR,'16 Ago, 2017','06 Mar, 2022')
to_df = pd.DataFrame(ohlcv)
clean_df = to_df.drop(columns=[0,6,7,8,9,10,11])
clean_df.columns = ['Open','High','Low','Close','Volume']
clean_df['Open'] = pd.to_numeric(clean_df['Open'])
clean_df['High'] = pd.to_numeric(clean_df['High'])
clean_df['Low'] = pd.to_numeric(clean_df['Low'])
clean_df['Close'] = pd.to_numeric(clean_df['Close'])
clean_df['Volume'] = pd.to_numeric(clean_df['Volume'])

clean_df.tail()

,Open,High,Low,Close,Volume
9950,2638.14,2657.79,2631.00,2648.19,48403.7664
9951,2648.18,2672.23,2627.34,2642.81,52112.0031
9952,2642.81,2685.12,2640.00,2670.71,40771.7549
9953,2670.72,2681.16,2650.69,2665.15,26683.8615
9954,2665.16,2676.29,2635.00,2639.26,26540.3981


In [8]:
class SMACross(Strategy):
    mm1 = 20
    mm2 = 30
    mm3 = 200

    def init(self):
        super().init()
        self.sma20 = self.I(SMA, self.data.Close, self.mm1)
        self.sma30 = self.I(SMA, self.data.Close, self.mm2)
        self.sma200 = self.I(SMA, self.data.Close, self.mm3)
    

    def next(self):
        super().next()
        stop_loss = 0.10
        take_profit = 0.20
        size = 10000 * 0.10
        price = clean_df['Close']
        engulfing_bearish = talib.CDLENGULFING(clean_df['Open'], clean_df['High'], clean_df['Low'], clean_df['Close'])
        engulfing_bullish = talib.CDLENGULFING(clean_df['Open'], clean_df['Low'], clean_df['High'], clean_df['Close'])
        

        if crossover(self.sma20, self.sma200):
            if price > self.sma20:
                self.buy()
    
        elif crossover(self.sma200, self.sma20):
            if price < self.sma20:
                self.sell()
            
bt = Backtest(clean_df,SMACross, cash = 10000, commission = 0.00, trade_on_close=False)
output = bt.run()
print(output)
bt.plot()


C:\Users\MI PC\AppData\Local\Temp\ipykernel_6948\2893332585.py:31: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(clean_df,SMACross, cash = 10000, commission = 0.00, trade_on_close=False)


ValueError: ('Lengths must match to compare', (9955,), (277,))

C:\Users\MI PC\AppData\Local\Temp\ipykernel_14780\4047231230.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(clean_df,SMACross, cash = 10000, commission = 0.05, exclusive_orders= True)


ValueError: ('Lengths must match to compare', (9955,), (201,))